### AS the data is huge we are building separate models by splitting w.r.t to city

In [1]:
import numpy as np
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')
from xgboost.sklearn import XGBRegressor

In [2]:
path=os.getcwd()
path

'C:\\Users\\jayam\\Downloads\\7879phd'

In [7]:
#Approach:-
#1) split the train_data, test_data with respect to city into seperate files

#model1: sales contribution model
#3) make copy of train_1 as train_1_contri, test_1 as test_contri
#6) create sales_contribution column in train_1_contri(groupby date)
#7) predict sales contribution for train_contri and test_1_contri using xgboost
#8) store the predictions into new train_contri_predictions df and test_contri_predictions df
#9) creating test sales contribution sum by grouping w.r.t to date and getting sales contribution sum per day 
#10) so, basically we are creating new dataframe test_sale_contri_sum 
#11)  merge with test_contri_predictions to get actual sales contribution in test dataset

#model2: footfall model
#8) create a model for predicting footfall data
#9) use xgboost to predict footfall for test_1_total_sales data
#10) make foofall_1_test df from the prediction for test_1_total_sales

#model3: total sales model
#11) make a copy of train_1 as train_1_total_sales and test_1 as test_1_total_sales
#12) groupby the train_1_total_sales wrt to year,month,day and aggregate the sales
#13) merge train_1_total_sales with footfall_1
#14) merge test_1_total_sales with footfall_1_test
#15) predict total sales for train_1_total_sales and test_1_total_sales using xgboost
#16) store the predictions into new train_1_total_sales_prediction df and test_1_total_sales_prediciton df

#output generator
#create a sales column in the train_contri_predictions by multiplying each row's sales contribution 
#to the total sales for that day taken from the train_1_total_sale_prediction 
#create a sales column in the test_contri_predictions by multiplying each row's sales contribution 
#to the total sales for that day taken from the test_1_total_sale_prediction 

#done 

In [8]:
def mean_absolute_percentage_error(y_true, y_pred): # function for getting mape value
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [9]:
# creation a function which takes X_train,y_train and X_test as inputs 
def model(X_train,y_train,X_test):# creating a function for modelling which returns test predictions and its mape value
    clf=XGBRegressor(njobs=-1,seed=123)# we are experimenting only with 1 model xgboost as base model
    clf = clf.fit(X_train, y_train)
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    mape=mean_absolute_percentage_error(y_train, train_pred)# calculate mape
    return test_pred,mape

In [10]:
def merge_fun(df1,df2,cols,join_type):# function for merging
    return df1.merge(df2,on=cols,how=join_type)

In [11]:
def model_function(files):
    df=pd.DataFrame()
    for file in files:
        # fixing the columns for sales contribution prediction for modelling
        sales_contri_cols=["year","month","day","medicine","discounted","week"]
        file=str(file)
        city=file
        train=pd.read_csv("train_"+file+".csv")# read each train file
        test=pd.read_csv("test_"+file+".csv")# read each test file
        train["discounted"]=train["discounted"].fillna(0)# fill na values for discounted column
        train_contri=train.copy()# creating a copy of train dataset
        test_contri=test.copy()# creating a copy of test dataset
        # creating sales sum dataframe
        sale_sum=train_contri.groupby("date")["sales"].sum().reset_index()# grouping with date and take sum of  sales
        sale_sum.rename(columns={"sales":"sales_sum"},inplace=True)# renaming sales column as salesum
        # merging sales sum with train dataset
        train_contri=merge_fun(train_contri,sale_sum,["date"],"left")# merge sales with with train_contri
        # creating sales contribution
        train_contri["sales_contribution"]=train_contri["sales"]/train_contri["sales_sum"]
        # passing to data to model 
        sales_contri=model(train_contri[sales_contri_cols],train_contri["sales_contribution"],test_contri[sales_contri_cols])
        print("sales contribution mape of city-",file,"is",sales_contri[1])
        # storing test sales_contribution in test_contri_predictions
        test_contri_predictions=test_contri[["date","id"]]
        test_contri_predictions["sales_contribution"]=sales_contri[0]
        
        # creating test sales conrribution sum by grouping w.r.t to date and getting sales contribution sum per day 
        # basically we are creating new dataframe test_sale_contri_sum 
        # merge with test_contri_predictions to get actual sales contribution in test dataset
        
        test_sale_contri_sum=test_contri_predictions.groupby(["date"])["sales_contribution"].sum().reset_index()
        #creating sales contribution sum dataframe
        test_sale_contri_sum.rename(columns={"sales_contribution":"sales_contribution_sumperday"},inplace=True)
        #merge with test contri predictions
        test_contri_predictions=test_contri_predictions.merge(test_sale_contri_sum,on=["date"],how="left")
        
        # creating a column sales contributions contribution by dividing sales contribution with sum of sales contribution per day 
        # we are creating this column because sum of sales contribution is not 1 per day
        
        test_contri_predictions["sales_contributions_contribution"]=test_contri_predictions["sales_contribution"]/test_contri_predictions["sales_contribution_sumperday"]
        # drop sales_contribution_sumperday and sales_contribution
        test_contri_predictions.drop(["sales_contribution_sumperday","sales_contribution"],axis=1,inplace=True)
        
        # build model for predicting footfall 
        # creating x_train,y_train and x_test for footfall model
        X_train_footfall=train_contri[["year","month","week","day"]].drop_duplicates().reset_index(drop=True)
        X_test_footfall=test_contri[["year","month","week","day"]].drop_duplicates().reset_index(drop=True)
        y_train_footfall=train_contri[["date","footfall"]].drop_duplicates().reset_index(drop=True)
        y_train_footfall=y_train_footfall["footfall"]
        
        #print( X_train_footfall.shape,X_train_footfall.head(2))
        #print( X_test_footfall.shape)
        #print( y_train_footfall.shape)
        # pass the data to footfall model
        
        footfall_model=model(X_train_footfall,y_train_footfall,X_test_footfall)
        print("footfall mape of city-",file,"is",footfall_model[1])
        # storing footfall predictions
        test_footfall_predictions=test_contri[["year","month","week","day"]].drop_duplicates().reset_index(drop=True)
        test_footfall_predictions["footfall"]=np.round(footfall_model[0])
        
        # build a model for predicting total sales
        # create a dataframe for storing sum of sales
        train_total_sales=train.groupby(["date"])["sales"].sum().reset_index()
        # storing footfall data only for train data set
        footfall=train[["date","year","month","week","day","footfall"]].drop_duplicates().reset_index(drop=True)
        
        #merge the total sales data with footfall
        train_total_sales=merge_fun(train_total_sales,footfall,["date"],"left")
        
        # creating a dataset for test data for total sales
        test_total_sales=test[["date","year","month","week","day"]].drop_duplicates().reset_index(drop=True)
        
        # merge test total sales with footfall
        test_total_sales=merge_fun(test_total_sales,test_footfall_predictions,["year","month","week","day"],"left")
        
        # creating x_train,y_train and x_test for total sales model
        
        X_train_sales=train_total_sales.drop(["date","sales"],axis=1)
        X_test_sales=test_total_sales.drop(["date"],axis=1)
        X_train_sales= X_train_sales[['footfall', 'year', 'month', 'week', 'day']]
        X_test_sales=X_test_sales[['footfall', 'year', 'month', 'week', 'day']]
        y_train_sales=train_total_sales["sales"]
        
        # pass the data to sales_model for getting total sales predictions
        sales_model=model(X_train_sales,y_train_sales,X_test_sales)
        print("total sales  mape of city-",file,"is",sales_model[1])
        
        test_total_sales_prediciton=test_total_sales[["date"]]
        test_total_sales_prediciton["total_sales"]=sales_model[0]
        
        # now we have total sales data and sales contribution data
        # sales contribution= sales /total sales
        # so, sales = total sales *sales contribution
        
        test_contri_predictions=merge_fun(test_contri_predictions,test_total_sales_prediciton,["date"],"left")
        test_contri_predictions["sales"]=test_contri_predictions["sales_contributions_contribution"]*test_contri_predictions["total_sales"]
        
        # drop total_sales,sales_contributions and store test predictions
        test_contri_predictions.drop(["total_sales","sales_contributions_contribution"],axis=1,inplace=True)
        
        # concat all the test predictions for all the cities
        df=pd.concat([df,test_contri_predictions])
        
    return df
        

In [17]:
list(range(1,11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [18]:
pred=model_function(list(range(1,11)))

[09:28:51] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
sales contribution mape of city- 1 is 255.2120242646947
[09:31:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
footfall mape of city- 1 is 8.128923716249714
[09:31:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
total sales  mape of city- 1 is 9.258750554745195
[09:31:20] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
sales contribution mape of city- 2 is 183.21571828035536
[09:32:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated i

In [19]:
pred.to_csv("predictions_ver3.csv",index=False)

#### Conclusion

####  This model is giving score of 76.54 in sct so, this is not a good approach, may be model is missing relationship between cities, so let us build a model on the entire dataset 